In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install seqeval


     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=4c99528ac4e2288db9f75a56aeab638115aea9800e144bbc10652c2981e8a8a1
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from crf import CRF 

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/crf.py')
!cp '/content/drive/My Drive/crf.py' .

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 6.0MB/s 
     |████████████████████████████████| 2.9MB 57.6MB/s 
     |████████████████████████████████| 890kB 56.0MB/s 
     |████████████████████████████████| 1.1MB 55.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=989ced61c866f2bbcb93e990a9b837b35930fee20e8e8fde5c733f282795f068
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:

import tensorflow as tf
from tensorflow.keras import layers

  
import numpy as np
import os
import time


from tensorflow.keras.models import Model, Sequential
from transformers import (
    AutoConfig,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    TFAutoModelForTokenClassification,
    TFTrainer,
    TFTrainingArguments,
    TFAutoModel,
)

print(tf.__version__)

2.3.0


In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('/content/drive/My Drive/outBIO.csv')

In [ ]:
data.drop(data.columns[[0]], axis=1, inplace=True)

In [ ]:
data = data.rename(columns={'sentence_no': 'Sentence', 'words': 'Word', 'NER_labels': 'NER_label'})

In [ ]:
labels_ner = list(set(data["NER_label"].values))
tag2idx = {t: i for i, t in enumerate(labels_ner)}
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

In [ ]:
train, test = train_test_split(data, test_size=0.1)

In [ ]:
train.size

1181616

In [ ]:
test.size

295407

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        tokenized_sentence.extend(tokenized_word)
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
  def get_bert_input(data,tokenizer,labels_ner):
    
    examples=[]
    sentence_word = []

    ner_sentence = []
    ner_labels = []

    grouped = data.groupby("Sentence")
    dataframes = [group for _, group in grouped]
    example = [[],[]]
    for dataframe in dataframes:
      example[0]=(dataframe["Word"].tolist())
      ner_sentence.append(['[CLS]']+dataframe["Word"].tolist()+['[SEP]'])

      example[1]=(dataframe["NER_label"].tolist())
      ner_labels.append(['O']+dataframe["NER_label"].tolist()+['O'])
      #sentence_word.append(word_indexing(dataframe["Word"].tolist(),vocab2id))
      #label.append(word_indexing(dataframe["NER_label"].tolist(),tag2id))
      examples.append(example)
      example=[[],[]]
    examples.sort()
    
    tokenized_texts_and_labels = [
        tokenize_and_preserve_labels(sent, labs)
        for sent, labs in zip(ner_sentence, ner_labels)
    ]

    tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
    labels_texts = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]
    max_len = 100
    attention_mask = [[1]*len(sent)+[0]*(max_len-len(sent)) for sent in tokenized_texts]
    tags = tf.keras.preprocessing.sequence.pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_texts],
                     maxlen=100, padding="post",
                      truncating="post")
    y = [tf.keras.utils.to_categorical(i, num_classes=len(labels_ner)) for i in tags]
    sentence_mask = tf.keras.preprocessing.sequence.pad_sequences(attention_mask,
                              maxlen=100,truncating="post", padding="post", value = 0)
    input_ids = tf.keras.preprocessing.sequence.pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                              maxlen=100,truncating="post", padding="post")
    return [tf.cast(input_ids,tf.int32), tf.cast(sentence_mask,tf.int32)],tf.cast(y,tf.int32)

In [ ]:
x_tr,y_tr = get_bert_input(train,tokenizer,labels_ner)
x_te,y_te = get_bert_input(test,tokenizer,labels_ner)

In [ ]:
%load_ext tensorboard


In [ ]:
train_dataset1 = tf.data.Dataset.from_tensor_slices(
    ({'input_ids_in': input_ids, 'input_masks_in':attention_masks },y))

In [ ]:
from datetime import datetime
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=100,batch_size=32,write_graph=True, write_grads=True,write_images=True,embeddings_freq=100)

In [ ]:
input_ids_in = tf.keras.layers.Input(shape=(100,), name='input_token', dtype=tf.int32)
input_mask_in = tf.keras.layers.Input(shape=(100,), name='input_mask', dtype=tf.int32)
bert_model = TFAutoModel.from_pretrained("dbmdz/bert-base-turkish-cased")
embedding_layer = bert_model(input_ids_in, attention_mask = input_mask_in)[0]
model = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,trainable=False,
                          return_sequences=True))(embedding_layer)
model = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(labels_ner), activation="relu"))(model)
          
crf = CRF(len(labels_ner))  # CRF layer
out = crf(model)  # output
model = Model([input_ids_in,input_mask_in], out)
model.compile('adam', loss=crf.loss, metrics=[crf.accuracy])

print("Baseline/LSTM-CRF model built: ")
model.summary() 

Some layers from the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at dbmdz/bert-base-turkish-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Baseline/LSTM-CRF model built: 
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_1 (TFBertModel)   ((None, 100, 768), ( 110617344   input_token[0][0]                
                                                                 input_mask[0][0]                 
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 100)     32

In [ ]:
  
history = model.fit(x_tr, y_tr ,batch_size=32 ,epochs=3,verbose=1,validation_split = 0.1,callbacks=[tensorboard_callback])
test_pred = model.predict(x_te, verbose=1)
tag2idx = {t: i for i, t in enumerate(labels_ner)}
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

Epoch 1/3
775/775 [==============================] - 793s 1s/step - loss: 28.6741 - viterbi_accuracy: 0.8770 - val_loss: 147.1381 - val_viterbi_accuracy: 0.3357
Epoch 2/3
775/775 [==============================] - 769s 992ms/step - loss: 18.1246 - viterbi_accuracy: 0.9087 - val_loss: 127.7038 - val_viterbi_accuracy: 0.8363
Epoch 3/3
601/601 [==============================] - 180s 299ms/step
F1-score: 98.9%


In [ ]:
%tensorboard --logdir logs